In [ ]:
# Define dataset
SEQUENCE_NAME = "synthetic_bear"
CALIBRATION_NAME = "lightdome.json"

In [ ]:
# Imports and data loading
from modules.smvp_srv.data import *
import os
import logging as log
from IPython.display import Image

import numpy as np
import scipy
import cv2 as cv
from mpl_toolkits.mplot3d import Axes3D  
# Axes3D import has side effects, it enables using projection='3d' in add_subplot
import matplotlib.pyplot as plt

# Init logging
log.basicConfig(level='INFO', format='[%(levelname)s] %(message)s')

pi_by_2 = np.pi/2

# Load data
dataset_path = os.path.join(os.path.abspath("../HdM_BA/data/capture"), SEQUENCE_NAME)
calibration_path = os.path.join(os.path.abspath("../HdM_BA/data/calibration"), CALIBRATION_NAME)

seq = Sequence()
seq.loadFolder(dataset_path)

cal = Calibration(calibration_path)
lpseq = LpSequence(seq, cal)

In [ ]:
# Define Pixel coordinate (y, x) to plot data for
PIXEL_COORDS = ([918, 305], [650, 850])

# Get preview and show coordinate
img = seq.getPreview().asDomain(ImgDomain.sRGB, no_taich=True).asInt().get()
for coord in PIXEL_COORDS:
    img = cv.circle(img, coord, 20, [255,0,0], 5)
    img = cv.circle(img, coord, 1, [0,255,0], 1)

plt.figure(figsize=(20,20)) # New big figure
#plt.subplot(1,1,1) 
plt.imshow(img, interpolation='nearest')
# New figure
#fig_big = plt.figure(figsize=(10,10))
#plt.subplot(2,1,1) # nrows, ncols, plot_number
#plt.imshow(img, interpolation='nearest')
#plt.subplot(2,1,2) # nrows, ncols, plot_number
#plt.imshow(img, interpolation='nearest')

In [ ]:
## Define Data Plotter with coords from calibration

# Polar plot where top lights are in center
def plotData(data, trisurf, normalize=True, clip=True, ax=None, title=None):
    if ax is None:
        fig = plt.figure()
        ax = fig.add_subplot(projection='3d')
    if title is not None:
        ax.set_title(title)
    
    x, y, z = data
    
    # Normalize
    z_max = np.max(z)
    z_offset = -1.2 if normalize else -1.2 * z_max
    z_lim_max = 1.0 if normalize else z_max
    # Normalize & clip
    if normalize:
        z = z / z_max
    if clip:
        z = np.clip(z, 0, None)
        
    if trisurf:
        # Plot the surface.
        ax.plot_trisurf(x, y, z, vmin=z.min()*2, cmap=plt.cm.YlGnBu_r, antialiased=True)

        # Plot projections of the contour
        ax.tricontourf(x, y, z, zdir='z', offset=z_offset, cmap=plt.cm.coolwarm)

        # Set limits and add labels
        ax.set(xlim=(-0.8, 0.8), ylim=(-0.8, 0.8), zlim=(z_offset, z_lim_max),\
            xlabel='X', ylabel='Y', zlabel='Z')
    else:
        # Same as above
        ax.plot_surface(x, y, z, cmap=plt.cm.YlGnBu_r, antialiased=True)
        ax.contourf(x, y, z, zdir='z', offset=z_offset, cmap='coolwarm')
        ax.set(xlim=(-1.2, 1.2), ylim=(-1.2, 1.2), zlim=(z_offset, z_lim_max),\
            xlabel='X', ylabel='Y', zlabel='Z')

def plotGridData(data, normalize=True, clip=True, ax=None, title=None):
    plotData(data, False, normalize, clip, ax, title)
def plotPointData(data, normalize=True, clip=True, ax=None, title=None):
    plotData(data, True, normalize, clip, ax, title)



In [ ]:

def sampleSeqData(lp_sequence, pix, rotation_steps=0):
    # Create the mesh in polar coordinates and compute corresponding Z.
    x = np.zeros((0, ))
    y = np.zeros((0, ))
    z = np.ones((0, ))

    for _, img, lp in lp_sequence:
        ll = lp.getLL()
        lp_rot = LightPosition.FromLatLong([ll[0], ll[1]+rotation_steps*pi_by_2])
        x = np.append(x, [lp_rot.getZVecNorm()[0]])
        y = np.append(y, [lp_rot.getZVecNorm()[1]])
        z = np.append(z, [img.getPix(pix).lum().get()])
        
    return x, y, z

def sampleRtiData(fn, calibration, coord_system, rotation_steps=0):
    # Create the mesh in polar coordinates and compute corresponding Z.
    x = np.zeros((0, ))
    y = np.zeros((0, ))
    z = np.ones((0, ))
    
    for _, lp in calibration:
        ll = lp.getLL()
        lp_rot = LightPosition.FromLatLong([ll[0], ll[1]+rotation_steps*pi_by_2])
        x = np.append(x, [lp_rot.getZVecNorm()[0]])
        y = np.append(y, [lp_rot.getZVecNorm()[1]])
        # Fill arrays with coords and samples
        match coord_system:
            case CoordSys.LatLong:
                z = np.append(z, [fn(lp.getLLNorm())])
            case CoordSys.ZVec:
                z = np.append(z, [fn(lp.getZVecNorm())])
            case CoordSys.XYZ:
                z = np.append(z, [fn(lp.getXYZ())])
        
    return x, y, z
    

In [ ]:
## Define RTI fitter functions
from modules.smvp_srv.processing.fitter.spherical import *

# General PTM function
def calcPtm(coord, pix, seq):
    u, v = coord
    
    # 1.0 + lat + long
    val = seq[0].getPix(pix).lum().get() +\
    seq[1].getPix(pix).lum().get() * u
    seq[2].getPix(pix).lum().get() * v

    # Higher degrees
    idx = 3
    for n in range(2, 6+1): # 6 is max degree
        if idx >= len(seq):
            break
        
        for i in range(n+1):
            val += u**(n-i) * v**i * seq[idx].getPix(pix).lum().get()
            idx += 1
    
    return val

# Spherical Harmonics
def calcShm(coord, pix, seq):
    x,y,z = coord
    val = 0
    for i in range(len(seq)):
        l = math.floor(math.sqrt(i))
        m = i - l * (l + 1)
        # l & m are parameters of the degree of the harmonics in the shape of:
        # (0,0), (1,-1), (1,0), (1,1), (2,-2), (2,-1), ...
        #line[coeff_num] = SHFitter.SH(l, m, u, v)
        val += SHFitter.SHHardCoded(l, m, x, y, z) * seq[i].getPix(pix).lum().get()
        #val += scipy.special.sph_harm(l, m, (lat+1)*pi_by_2, (long+1)*np.pi)
        
    return val


## Define RTI Plotter

def sampleRtiGridData(fn, coord_system, rotation_steps=0, resolution=50):
    # Create the mesh in polar coordinates and compute corresponding Z.
    range_lat = np.linspace(1.0, -1.0, resolution, endpoint=False) # Normalized -np.pi/2, np.pi/2
    range_long = np.linspace(1.0, -1.0, resolution) # Normalized -np.pi, np.pi
    LAT, LONG = np.meshgrid(range_lat, range_long) # First is radius and repeats, second is same in whole array
    Z = np.empty((0, ))
    
    def getSampleArray(lat_arr, long_arr):
        samples = np.empty((0,))
        for ll in zip(lat_arr, long_arr):
            match coord_system:
                case CoordSys.LatLong:
                    samples = np.append(samples, fn(ll))
                case CoordSys.ZVec:
                    samples = np.append(samples, fn(LightPosition.FromLatLong(ll, normalized=True).getZVecNorm()))
                case CoordSys.XYZ:
                    samples = np.append(samples, fn(LightPosition.FromLatLong(ll, normalized=True).getXYZ()))
        return samples

    for lat, long in zip(LAT, LONG):
        if Z.shape == (0,):
            Z = getSampleArray(lat, long)
        else:
            Z = np.vstack((Z, getSampleArray(lat, long)))
        
    # Express the mesh in the cartesian system.
    X, Y = (-LAT+1)/2*np.sin(LONG*np.pi + rotation_steps*pi_by_2), -(-LAT+1)/2*np.cos(LONG*np.pi + rotation_steps*pi_by_2)
    return X, Y, Z
    



In [ ]:
# Get real data and plot
#plt.figure(figsize=(10,10))
# Sample sequence
seq_data1 = sampleSeqData(lpseq, PIXEL_COORDS[0], rotation_steps=2)
seq_data2 = sampleSeqData(lpseq, PIXEL_COORDS[1], rotation_steps=2)
# Sample PTM & PTMZ
ptm = seq.getDataSequence('ptm')
ptmz = seq.getDataSequence('ptmz')
shm = seq.getDataSequence('shm')
ptm_data1 = sampleRtiData(lambda coord: calcPtm(coord, PIXEL_COORDS[0], ptm), cal, CoordSys.LatLong, rotation_steps=2)
ptmz_data1 = sampleRtiData(lambda coord: calcPtm(coord, PIXEL_COORDS[0], ptmz), cal, CoordSys.ZVec, rotation_steps=2)
shm_data1 = sampleRtiData(lambda coord: calcShm(coord, PIXEL_COORDS[0], shm), cal, CoordSys.XYZ, rotation_steps=2)
ptm_data2 = sampleRtiData(lambda coord: calcPtm(coord, PIXEL_COORDS[1], ptm), cal, CoordSys.LatLong, rotation_steps=2)
ptmz_data2 = sampleRtiData(lambda coord: calcPtm(coord, PIXEL_COORDS[1], ptmz), cal, CoordSys.ZVec, rotation_steps=2)
shm_data2 = sampleRtiData(lambda coord: calcShm(coord, PIXEL_COORDS[1], shm), cal, CoordSys.XYZ, rotation_steps=2)

# Pixel 1
fig, axs = plt.subplots(ncols=4, figsize=(20,5), subplot_kw={'projection': '3d'}) # nrows=1
fig.suptitle('Pixel 1 data comparision')
plotPointData(seq_data1, normalize=False, ax=axs[0], title="Sequence")
plotPointData(ptm_data1, normalize=False, ax=axs[1], title="PTM")
plotPointData(ptmz_data1, normalize=False, ax=axs[2], title="PTMZ")
plotPointData(shm_data1, normalize=False, ax=axs[3], title="SMH")
plt.show()

# Pixel 2
fig, axs = plt.subplots(ncols=4, figsize=(20,5), subplot_kw={'projection': '3d'}) # nrows=1
fig.suptitle('Pixel 2 data comparision')
plotPointData(seq_data2, normalize=False, ax=axs[0], title="Sequence")
plotPointData(ptm_data2, normalize=False, ax=axs[1], title="PTM")
plotPointData(ptmz_data2, normalize=False, ax=axs[2], title="PTMZ")
plotPointData(shm_data2, normalize=False, ax=axs[3], title="SMH")
plt.show()


In [ ]:
# Get data sequence and plot
ptm = seq.getDataSequence('ptm')
ptmz = seq.getDataSequence('ptmz')
shm = seq.getDataSequence('shm')

ptm1_griddata = sampleRtiGridData(lambda coord: calcPtm(coord, PIXEL_COORDS[0], ptm), CoordSys.LatLong, rotation_steps=2)
ptm2_griddata = sampleRtiGridData(lambda coord: calcPtm(coord, PIXEL_COORDS[1], ptm), CoordSys.LatLong, rotation_steps=2)
ptmz1_griddata = sampleRtiGridData(lambda coord: calcPtm(coord, PIXEL_COORDS[0], ptmz), CoordSys.ZVec, rotation_steps=2)
ptmz2_griddata = sampleRtiGridData(lambda coord: calcPtm(coord, PIXEL_COORDS[1], ptmz), CoordSys.ZVec, rotation_steps=2)
shm1_griddata = sampleRtiGridData(lambda coord: calcShm(coord, PIXEL_COORDS[0], shm), CoordSys.XYZ, rotation_steps=2)
shm2_griddata = sampleRtiGridData(lambda coord: calcShm(coord, PIXEL_COORDS[1], shm), CoordSys.XYZ, rotation_steps=2)

# Pixel 1
fig, axs = plt.subplots(ncols=4, figsize=(20,5), subplot_kw={'projection': '3d'}) # nrows=1
fig.suptitle('Pixel 1 grid data comparision')
plotPointData(seq_data1, normalize=False, ax=axs[0], title="Sequence")
plotGridData(ptm1_griddata, normalize=True, ax=axs[1], title="PTM")
plotGridData(ptmz1_griddata, normalize=True, ax=axs[2], title="PTMZ")
plotGridData(shm1_griddata, normalize=True, ax=axs[3], title="SHM")
plt.show()

# Pixel 2
fig, axs = plt.subplots(ncols=4, figsize=(20,5), subplot_kw={'projection': '3d'}) # nrows=1
fig.suptitle('Pixel 2 grid data comparision')
plotPointData(seq_data2, normalize=False, ax=axs[0], title="Sequence")
plotGridData(ptm2_griddata, normalize=True, ax=axs[1], title="PTM")
plotGridData(ptmz2_griddata, normalize=True, ax=axs[2], title="PTMZ")
plotGridData(shm2_griddata, normalize=True, ax=axs[3], title="SHM")
plt.show()

